<br><br><br><br><br>

# More stuff!

(Based on requests from yesterday.)

<br><br><br><br><br>

### NumExpr

For "flat" expressions, like ufuncs, but can do many operations in one pass.

<center><img src="img/flat.png" width="25%"></center>

NumExpr compiles a limited set of expressions (mathematical formulae, no loops) for its own virtual machine. (Being a much simpler virtual machine than Python's, it can be evaluated more quickly.)

<br>

If you have Pandas installed, you have NumExpr. Pandas uses NumExpr when `apply` is given an expression in a string.

<br>

**Note:** I didn't find any documentation that says you can turn NumExpr expressions into ufuncs. (Shocking!) The author is working on NumExpr 3: I'll suggest it.

In [28]:
import numpy, numexpr

a = numpy.random.uniform(5, 10, 1000000)
b = numpy.random.uniform(10, 20, 1000000)
c = numpy.random.uniform(-0.1, 0.1, 1000000)

roots1 = (-b + numpy.sqrt(b**2 - 4*a*c)) / (2*a)

roots2 = numexpr.evaluate("(-b + sqrt(b**2 - 4*a*c)) / (2*a)")   # how does it know a, b, c?

roots2 = numexpr.evaluate("(-b + sqrt(b**2 - 4*a*c)) / (2*a)", global_dict={"a": a, "b": b, "c": c})

roots2 = numexpr.evaluate("(-b + sqrt(b**2 - 4*a*c)) / (2*a)", global_dict=globals())

print((roots1 == roots2).all())

True


In [9]:
%%timeit

roots1 = (-b + numpy.sqrt(b**2 - 4*a*c)) / (2*a)

12 ms ± 64.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit

roots2 = numexpr.evaluate("(-b + sqrt(b**2 - 4*a*c)) / (2*a)")

1.83 ms ± 31.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [33]:
import numba

@numba.jit(nopython=True)
def roots_numba(a, b, c):
    out = numpy.empty_like(a)
    for i in range(len(a)):
        out[i] = (-b[i] + numpy.sqrt(b[i]**2 - 4*a[i]*c[i])) / (2*a[i])
    return out

roots3 = roots_numba(a, b, c)

print(numpy.allclose(roots1, roots3))

True


In [34]:
%%timeit

roots3 = roots_numba(a, b, c)

2.62 ms ± 24.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
import numba

@numba.jit(nopython=True, fastmath=True)
def roots_numba(a, b, c):
    out = numpy.empty_like(a)
    for i in range(len(a)):
        out[i] = (-b[i] + numpy.sqrt(b[i]**2 - 4*a[i]*c[i])) / (2*a[i])
    return out

roots3 = roots_numba(a, b, c)

print(numpy.allclose(roots1, roots3))

True


In [37]:
%%timeit

roots3 = roots_numba(a, b, c)

2.62 ms ± 28.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
